Import packages 

In [2]:
import re 
import string
import spacy
import csv
import tqdm 
import pandas as pd
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

Job description text cleaning functions 

In [3]:
def clean_data(doc):
    """
    Received a spacy doc object and cleans it
    """
    doc = remove_stopwords(doc)
    doc = remove_punctuation(doc)
    doc = remove_numbers(doc)
    doc = remove_nonalphabeticals(doc)
    doc = convert_to_str(doc)
    
    return doc

def remove_stopwords(doc):
    """ removes stopwords from a doc """
    return [token for token in doc if not token.is_stop]

def remove_punctuation(doc):
    """ removes punctuation """
    return [token for token in doc if not token.is_punct]

def remove_numbers(doc):
    """ removes 'numbers' """
    return [token for token in doc if not token.like_num]

def remove_nonalphabeticals(doc):
    """ removes non-alphabeticals """
    return [token for token in doc if token.is_alpha]

def remove_entities(doc, ents = ['PERSON', 'ORG', 'GPE']):
    return [token for token in doc if token.ent_type_ not in ents]

def convert_to_str(doc, lemmatize = True):
    """ convert spacy doc objects to string """
    return [token.lemma_.lower() if lemmatize else token.text.lower() for token in doc]

nlp = spacy.load('en_core_web_sm')

Read data from local .csv file 

In [4]:
jd_title = []
jd_year_exp = []
jd_text = []
with open ('calc_scrape_final.csv') as df: 
    jd = csv.reader(df)
    next(jd)
    for i in jd: 
        jd_title.append((i[9]))
        jd_year_exp.append((i[11]))
        jd_text.append((i[18]))

allwyn_title = []
allwyn_year = []
allwyn_jd_text = []
with open('Allwyn Initial Job Description Input .csv') as df: 
    a_jd = csv.reader(df)
    next(a_jd)
    for i in a_jd: 
        allwyn_title.append((i[1]))
        allwyn_year.append((i[3]))
        allwyn_jd_text.append((i[6])) 

FileNotFoundError: [Errno 2] No such file or directory: 'calc_scrape_final.csv'

Job description text cleaning 

In [ ]:
c_docs = nlp.pipe(jd_text) # text_corpus is simply a list of lists
calc_jd = [clean_data(doc) for doc in c_docs] # the entire data, cleaned 

a_docs = nlp.pipe(allwyn_jd_text)
clean_allwyn_jd = [clean_data(doc) for doc in a_docs] 

Create pendas dataframe with clean job descriptions 

In [ ]:
calc_rd = {"c_job_title": pd.Series(jd_title, dtype="string"), 
           "c_yesrs_of_experience": pd.Series(jd_year_exp), 
           "c_jd": pd.Series(calc_jd, dtype="string")}
calc_df = pd.DataFrame(calc_rd)

a_rd = {"a_job_title": pd.Series(allwyn_title, dtype="string"), 
        "a_yesrs_of_experience": pd.Series(allwyn_year), 
        "a_jd": pd.Series(clean_allwyn_jd, dtype="string")}
a_df = pd.DataFrame(a_rd) 

Job description matching without using pendas dataframe 

In [1]:
ac_list0 = []
ac_list_count0 = []
#def exact_match(calc_jd, clean_allwyn_jd): 
for i in range(len(calc_jd)): 
    a1 = clean_allwyn_jd[0]
    s1 = set(calc_jd[i])
    s2 = set(a1)
    ac2 = list(s1.intersection(s2))
    ac_list0 .append(ac2)
    ac_count2 = len(ac2)
    ac_list_count0.append(ac_count2)

NameError: name 'calc_jd' is not defined